
# Spark — Reviews con **UDF** (Windows-friendly)

Este notebook carga un CSV de reseñas (`reviews.csv`) y usa una **UDF de PySpark** para contar palabras por fila.  
Incluye ajustes para Windows que evitan el error `Python worker failed to connect back`.



## 0) (Windows) Usar el mismo Python para driver y ejecutores

Ejecuta esta celda **antes** de crear la `SparkSession`.


In [13]:

import os, sys
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
print("PYSPARK_PYTHON =", os.environ.get("PYSPARK_PYTHON"))
print("PYSPARK_DRIVER_PYTHON =", os.environ.get("PYSPARK_DRIVER_PYTHON"))


PYSPARK_PYTHON = c:\Users\patri\AppData\Local\Programs\Python\Python310\python.exe
PYSPARK_DRIVER_PYTHON = c:\Users\patri\AppData\Local\Programs\Python\Python310\python.exe


## 1) Crear SparkSession

In [14]:

from pyspark.sql import SparkSession
import sys

spark = (SparkSession.builder
         .appName("UDF_Reviews_Windows")
         # Evita problemas con antivirus/firewall en Windows
         .config("spark.python.use.daemon","false")
         # Propaga el mismo intérprete de Python a los ejecutores
         .config("spark.executorEnv.PYSPARK_PYTHON", sys.executable)
         .getOrCreate())

spark



## 2) Cargar `reviews.csv` (o generar uno de ejemplo)

- Estructura mínima esperada: `review_id, review_text`.  
- Si el archivo no existe en la ruta indicada, se crea un CSV de ejemplo.


In [15]:

from pathlib import Path
import csv

input_path = Path("reviews.csv")

if not input_path.exists():
    rows = [
        ("r001", "Excelente servicio y rápida entrega. Muy recomendado."),
        ("r002", "El producto llegó dañado, pero el soporte respondió a tiempo."),
        ("r003", "Calidad precio adecuado. Volvería a comprar."),
        ("r004", "No cumplió mis expectativas."),
        ("r005", "Todo bien."),
        ("r006", "Atención al cliente excepcional y envío en 24 horas."),
        ("r007", "No me gustó el empaque; recomendaría mejorar."),
        ("r008", "Muy útil, fácil de usar y con buen manual."),
        ("r009", "Precio alto para lo que ofrece."),
        ("r010", "Cinco estrellas. Superó mis expectativas."),
        ("r011", "Malo."),
        ("r012", "Regular. Se puede mejorar."),
        ("r013", "Entrega tardía, pero producto excelente."),
        ("r014", "Bonito diseño y buena batería."),
        ("r015", "Demasiado pesado para uso diario."),
        ("r016", "Funciona como se esperaba."),
        ("r017", "No funciona con mi sistema operativo."),
        ("r018", "La app companion es muy buena."),
        ("r019", "Me encantó, comprare otra unidad."),
        ("r020", "Servicio técnico resolvió mi problema en minutos.")
    ]
    with input_path.open("w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["review_id","review_text"])
        w.writerows(rows)
    print("Se creó un CSV de ejemplo en", input_path.resolve())

# Cargar el CSV con Spark
df = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv(str(input_path)))

df.show(5, truncate=False)
df.printSchema()


+---------+----+-----------------------------------+------+
|review_id|user|review_text                        |rating|
+---------+----+-----------------------------------+------+
|1        |u01 |Excelente producto, muy recomendado|5     |
|2        |u02 |Malo, se rompió a los dos días     |1     |
|3        |u03 |Cumple con lo esperado             |4     |
|4        |u04 |No me gustó la calidad del material|2     |
|5        |u05 |Precio justo y buena atención      |5     |
+---------+----+-----------------------------------+------+

root
 |-- review_id: integer (nullable = true)
 |-- user: string (nullable = true)
 |-- review_text: string (nullable = true)
 |-- rating: integer (nullable = true)




## 3) Definir **UDF** `contar_palabras` y aplicarla

La UDF corre en proceso Python por partición. Con la configuración anterior, debería funcionar correctamente en Windows.


In [16]:

from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType

def contar_palabras(s: str) -> int:
    if s is None:
        return 0
    # separa por espacios, ignora tokens vacíos
    return len([t for t in s.split() if t.strip()])

udf_contar_palabras = udf(contar_palabras, IntegerType())

df2 = df.withColumn("num_palabras", udf_contar_palabras(col("review_text")))
df2.show(truncate=False)


+---------+----+-----------------------------------+------+------------+
|review_id|user|review_text                        |rating|num_palabras|
+---------+----+-----------------------------------+------+------------+
|1        |u01 |Excelente producto, muy recomendado|5     |4           |
|2        |u02 |Malo, se rompió a los dos días     |1     |7           |
|3        |u03 |Cumple con lo esperado             |4     |4           |
|4        |u04 |No me gustó la calidad del material|2     |7           |
|5        |u05 |Precio justo y buena atención      |5     |5           |
+---------+----+-----------------------------------+------+------------+




## 4) (Opcional) UDF que detecta presencia de palabras clave

Ejemplo: marca `1` si encuentra alguna palabra de una lista de **keywords**.


In [17]:

from typing import List

KEYWORDS = {"excelente","malo","tardía","rápida","recomendado","soporte","dañado","pesado"}

def contiene_keywords(s: str) -> int:
    if not s:
        return 0
    txt = s.lower()
    return 1 if any(k in txt for k in KEYWORDS) else 0

from pyspark.sql.types import ShortType
udf_contiene_keywords = udf(contiene_keywords, ShortType())

df3 = df2.withColumn("flag_keywords", udf_contiene_keywords(col("review_text")))
df3.show(truncate=False)


+---------+----+-----------------------------------+------+------------+-------------+
|review_id|user|review_text                        |rating|num_palabras|flag_keywords|
+---------+----+-----------------------------------+------+------------+-------------+
|1        |u01 |Excelente producto, muy recomendado|5     |4           |1            |
|2        |u02 |Malo, se rompió a los dos días     |1     |7           |1            |
|3        |u03 |Cumple con lo esperado             |4     |4           |0            |
|4        |u04 |No me gustó la calidad del material|2     |7           |0            |
|5        |u05 |Precio justo y buena atención      |5     |5           |0            |
+---------+----+-----------------------------------+------+------------+-------------+

